Copyright (c) Microsoft Corporation. All rights reserved.

Licensed under the MIT License.

# Distributed Chainer using Datastore
In this tutorial, you will run a Chainer training example on the [MNIST](http://yann.lecun.com/exdb/mnist/) dataset using ChainerMN distributed training across a GPU cluster. Training data is uploaded during the notebook to Azure Blob storage and registered as "Datastore" in Azure Machine Learning service Workspace. You can use "Datastore" after this experiment to access to the data in Azure Blob storage.

## Prerequisites
* Go through the [Configuration](../../../configuration.ipynb) notebook to install the Azure Machine Learning Python SDK and create an Azure ML `Workspace`

In [75]:
# Check core SDK version number
import azureml.core

print("SDK version:", azureml.core.VERSION)

SDK version: 1.0.15


## Diagnostics
Opt-in diagnostics for better experience, quality, and security of future releases.

In [76]:
from azureml.telemetry import set_diagnostics_collection

set_diagnostics_collection(send_diagnostics=True)

Turning diagnostics collection on. 


## Initialize workspace

Initialize a [Workspace](https://docs.microsoft.com/azure/machine-learning/service/concept-azure-machine-learning-architecture#workspace) object from the existing workspace you created in the Prerequisites step. `Workspace.from_config()` creates a workspace object from the details stored in `config.json`.

In [77]:
from azureml.core.workspace import Workspace

ws = Workspace.get(name='azureml', 
                      subscription_id='9c0f91b8-eb2f-484c-979c-15848c098a6b', 
                      resource_group='amlservice'
                   )
print('Workspace name: ' + ws.name, 
      'Azure region: ' + ws.location, 
      'Subscription id: ' + ws.subscription_id, 
      'Resource group: ' + ws.resource_group, sep = '\n')

Workspace name: azureml
Azure region: southeastasia
Subscription id: 9c0f91b8-eb2f-484c-979c-15848c098a6b
Resource group: amlservice


## Create or attach existing AmlCompute
You will need to create a [compute target](https://docs.microsoft.com/azure/machine-learning/service/concept-azure-machine-learning-architecture#compute-target) for training your model. In this tutorial, we use Azure ML managed compute ([AmlCompute](https://docs.microsoft.com/azure/machine-learning/service/how-to-set-up-training-targets#amlcompute)) for our remote training compute resource. Specifically, the below code creates an `STANDARD_NC6` GPU cluster that autoscales from `0` to `4` nodes.

**Creation of AmlCompute takes approximately 5 minutes.** If the AmlCompute with that name is already in your workspace, this code will skip the creation process.

As with other Azure services, there are limits on certain resources (e.g. AmlCompute) associated with the Azure Machine Learning service. Please read [this article](https://docs.microsoft.com/en-us/azure/machine-learning/service/how-to-manage-quotas) on the default limits and how to request more quota.

In [78]:
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException

# choose a name for your cluster
cluster_name = "gpucluster"

try:
    compute_target = ComputeTarget(workspace=ws, name=cluster_name)
    print('Found existing compute target.')
except ComputeTargetException:
    print('Creating a new compute target...')
    compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_NC6',
                                                           max_nodes=4)

    # create the cluster
    compute_target = ComputeTarget.create(ws, cluster_name, compute_config)

    compute_target.wait_for_completion(show_output=True)

# use get_status() to get a detailed status for the current AmlCompute. 
print(compute_target.get_status().serialize())

Found existing compute target.
{'allocationState': 'Steady', 'allocationStateTransitionTime': '2019-02-21T02:53:37.348000+00:00', 'creationTime': '2019-02-20T14:25:23.447852+00:00', 'currentNodeCount': 2, 'errors': None, 'modifiedTime': '2019-02-20T14:25:56.236703+00:00', 'nodeStateCounts': {'idleNodeCount': 2, 'leavingNodeCount': 0, 'preemptedNodeCount': 0, 'preparingNodeCount': 0, 'runningNodeCount': 0, 'unusableNodeCount': 0}, 'provisioningState': 'Succeeded', 'provisioningStateTransitionTime': None, 'scaleSettings': {'minNodeCount': 0, 'maxNodeCount': 4, 'nodeIdleTimeBeforeScaleDown': 'PT120S'}, 'targetNodeCount': 2, 'vmPriority': 'Dedicated', 'vmSize': 'STANDARD_NC6S_V3'}


The above code creates GPU compute. If you instead want to create CPU compute, provide a different VM size to the `vm_size` parameter, such as `STANDARD_D2_V2`.

## Train model on the remote compute
Now that we have the AmlCompute ready to go, let's run our distributed training job.

### Create a project directory
Create a directory that will contain all the necessary code from your local machine that you will need access to on the remote resource. This includes the training script and any additional files your training script depends on.

In [89]:
import os

project_folder = './chainer-distr'
os.makedirs(project_folder, exist_ok=True)

### Prepare training script
Now you will need to create your training script. In this tutorial, the script for distributed training of MNIST is already provided for you at `train_mnist.py`. In practice, you should be able to take any custom Chainer training script as is and run it with Azure ML without having to modify your code.

Once your script is ready, copy the training script `train_mnist.py` into the project directory.

In [90]:
import shutil

shutil.copy('chainer_mnist.py', project_folder)

'./chainer-distr/chainer_mnist.py'

### Download MNIST Data

In [81]:
import urllib
import os

os.makedirs('./data/mnist', exist_ok=True)

urllib.request.urlretrieve('http://yann.lecun.com/exdb/mnist/train-images-idx3-ubyte.gz', filename='./data/mnist/train-images.gz')
urllib.request.urlretrieve('http://yann.lecun.com/exdb/mnist/train-labels-idx1-ubyte.gz', filename='./data/mnist/train-labels.gz')
urllib.request.urlretrieve('http://yann.lecun.com/exdb/mnist/t10k-images-idx3-ubyte.gz', filename='./data/mnist/test-images.gz')
urllib.request.urlretrieve('http://yann.lecun.com/exdb/mnist/t10k-labels-idx1-ubyte.gz', filename='./data/mnist/test-labels.gz')

('./data/mnist/test-labels.gz', <http.client.HTTPMessage at 0x117553630>)

### Upload MNIST data to default Blob

In [82]:
ds = ws.get_default_datastore()

In [83]:
ds.upload(src_dir='./data/mnist', target_path='mnist', overwrite=True, show_progress=True)

Uploading ./data/mnist/test-images.gz
Uploading ./data/mnist/test-labels.gz
Uploading ./data/mnist/train-images.gz
Uploading ./data/mnist/train-labels.gz
Uploaded ./data/mnist/test-labels.gz, 1 files out of an estimated total of 4
Uploaded ./data/mnist/train-labels.gz, 2 files out of an estimated total of 4
Uploaded ./data/mnist/test-images.gz, 3 files out of an estimated total of 4
Uploaded ./data/mnist/train-images.gz, 4 files out of an estimated total of 4


$AZUREML_DATAREFERENCE_3b38643741be42a284036a1b149fc0cd

Azure Portal もしくは、Storage ExplorerなどのツールからBlobにデータがアップロードされていることを確認します。



<div align="left"><img src= "../images/defaultblob.png" width="500" align ="left">

### Create an experiment
Create an [Experiment](https://docs.microsoft.com/azure/machine-learning/service/concept-azure-machine-learning-architecture#experiment) to track all the runs in your workspace for this distributed Chainer tutorial. 

In [91]:
from azureml.core import Experiment

experiment_name = 'chainer-distr'
experiment = Experiment(ws, name=experiment_name)

### Create a Chainer estimator
The Azure ML SDK's Chainer estimator enables you to easily submit Chainer training jobs for both single-node and distributed runs.

In [92]:
from azureml.train.dnn import Chainer

script_params = {
    '--data-folder': ds.path('mnist').as_mount()
}


estimator = Chainer(source_directory=project_folder,
                    compute_target=compute_target,
                    entry_script='chainer_mnist.py',
                    script_params = script_params,
                    pip_packages=['pytest'],
                    node_count=2,
                    process_count_per_node=1,
                    distributed_backend='mpi',
                    use_gpu=True)

The above code specifies that we will run our training script on `2` nodes, with one worker per node. In order to execute a distributed run using MPI, you must provide the argument `distributed_backend='mpi'`. Using this estimator with these settings, Chainer and its dependencies will be installed for you. However, if your script also uses other packages, make sure to install them via the `Chainer` constructor's `pip_packages` or `conda_packages` parameters.

### Submit job
Run your experiment by submitting your estimator object. Note that this call is asynchronous.

In [93]:
run = experiment.submit(estimator)
print(run)

Run(Experiment: chainer-distr,
Id: chainer-distr_1550718150_f9da8082,
Type: azureml.scriptrun,
Status: Queued)


### Monitor your run
You can monitor the progress of the run with a Jupyter widget. Like the run submission, the widget is asynchronous and provides live updates every 10-15 seconds until the job completes. You can see that the widget automatically plots and visualizes the loss metric that we logged to the Azure ML run.

In [94]:
from azureml.widgets import RunDetails

RunDetails(run).show()

_UserRunWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': True, 'log_level': 'INFO', 's…

In [95]:
run.wait_for_completion(show_output=True)

RunId: chainer-distr_1550718150_f9da8082

Execution Summary
RunId: chainer-distr_1550718150_f9da8082



{'runId': 'chainer-distr_1550718150_f9da8082',
 'target': 'gpucluster',
 'status': 'Completed',
 'startTimeUtc': '2019-02-21T03:02:51.772775Z',
 'endTimeUtc': '2019-02-21T03:03:57.810312Z',
 'properties': {'azureml.runsource': 'experiment',
  'ContentSnapshotId': 'c28211c7-a162-40df-8172-d0653af0b44c'},
 'runDefinition': {'Script': 'chainer_mnist.py',
  'Arguments': ['--data-folder',
   '$AZUREML_DATAREFERENCE_280a911219fe46e49d1b50ba286a1ed7'],
  'SourceDirectoryDataStore': 'workspaceblobstore',
  'Framework': 0,
  'Communicator': 5,
  'Target': 'gpucluster',
  'DataReferences': {'280a911219fe46e49d1b50ba286a1ed7': {'DataStoreName': 'workspaceblobstore',
    'Mode': 'Mount',
    'PathOnDataStore': 'mnist',
    'PathOnCompute': None,
    'Overwrite': False},
   'workspaceblobstore': {'DataStoreName': 'workspaceblobstore',
    'Mode': 'Mount',
    'PathOnDataStore': None,
    'PathOnCompute': None,
    'Overwrite': False}},
  'JobName': None,
  'AutoPrepareEnvironment': True,
  'MaxRunD